### ДЗ 5: WSD

#### 1) Код - здесь ниже
* Vectorizers: Count, Tf/idf
* Classifiers: multinomial naive Bayes, Random Forest, Decision Tree

#### 2) Данные:
https://github.com/phuuda/nlp_year4/blob/master/hw5_wsd/castle-lock.txt <br><br>

Данные + 10 собственных примеров:<br>
https://github.com/phuuda/nlp_year4/blob/master/hw5_wsd/test_contexts.txt

#### 3) Описание:

Данные:

<b>а.</b> Source: Sketch Engine, ruTenTen11, lemma searched = 'замок'

<b>б.</b><br>
* 700 контекстов употребления 'замок' = castle -- class 1
* 700 контекстов употребления 'замок' = lock -- class 0
* тексты ruTenTen11 -- из интернета
* значение лексемы в каждом примере размечалось вручную
* контексты -- примено 10-15 слов

<b>в.</b><br>
* лемматизатор -- Mystem
* стоп-лист -- расширенный список на основе Вордстата: http://www.bukvarix.com/bukvarix-stop-words.html<br>

<b>г.</b> примеры ошибок для модели <b>Initial vectors + Random Forest + Tf idf</b>:<br>
* <i>ночью замок был освещен и был открыт для посетителей и ночных экскурсий</i> (неясно, почему ошибочно классифицирует как 'lock'. видимо, не достаточно key-words про средневековье (<i>рыцари</i>, <i>старинный</i>, <i>королевский</i>, etc.))
* <i>ребенок пытался вставить карандаш в скважину и громко орал</i> (тоже не очень ясно, видимо из-за отсутствия слов <i>дверь</i>, <i>ключ</i>, <i>металл</i>, and similar.)

<b>д.</b> таблица с результатами по 6 моделям:<br>
<img src="https://raw.githubusercontent.com/phuuda/nlp_year4/master/hw5_wsd/hw5_wsd_results.png"><br>

* Для initial & reduced vectors Pipeline давал estimation, что <b>naive Bayes</b> + Count Vectorizer выдаст наибольший accuracy (89% и 82% respectively)<br><br>

* Для reduced vectors это частично подтвердилось: <b>Bayes</b> выдавал наибольший accuracy of classification, совпадающий у Tf/idf & Count Vectorizers (85%).<br><br>

* Однако, для initial (non-reduced) vectors <b>Random Forest</b> опережает остальные модели (90% accuracy score).<br><br>

* Интересно, что при inital vectors <b>Random Forest, Decision Tree</b>, и <b>Bayes</b> справляются довольно неплохо (81-90% accuracy), а для reduced vectors <b>Random Forest</b> и <b>Decision</b> резко ухудшают свои результаты (58-73% accuracy). В то же время, при reduced vectors <b>Bayes</b> работает одинаково хорошо при Tf/idf & Count Vectorizers (85% accuracy).<br><br>

* В среднем, Tf/idf дает лучшие результаты для 3 моделей.<br><br>

<b>е.</b> по 5 примеров ваших собственных на каждое значение (не из датасета) - результат применения к ним лучшего метода - <b>Random Forest + Tf/idf + Initial vectors</b><br><br>
```
1  они объездили всю Европу чтобы посетить самые знаменитые средневековые замки
1  ночью замок был освещен и был открыт для посетителей и ночных экскурсий
1  в подвалах и башнях старого замка было жутко холодно темно и страшновато
1  чтобы завоевать замок нужно было штурмовать еще и ближайшие деревни
1  в одной из башен замка освещенное окошко служило маяком слабой надежды
0  я могу взломать любой замок особенно если нужен не ключ а числовая комбинация
0  хозяева установили новые металлические замки с системой сигнализации
0  я не заметил замка на двери и врезался в нее почти с разбега
0  компания предлагает большой выбор замков оптом и с гарантией на четыре года
0  ребенок пытался вставить карандаш в скважину замка и громко орал

```
<b>Результат: 8/10</b> (ошибочная классификация примеров 2 и 10):
```
'ночью замок был освещен и был открыт для посетителей и ночных экскурсий'
'ребенок пытался вставить карандаш в скважину замка и громко орал'
```

In [415]:
import re
from collections import Counter
import os
from pprint import pprint
from pymystem3 import Mystem

In [416]:
m = Mystem()

### Загрузка стоп-слов

In [417]:
def load_stop_words(stop_filename):
	''' загрузить список стоп-слов из файла, одно слово на строке '''
	with open(stop_filename, encoding = 'utf-8') as f:
		stopwords = [w.strip() for w in f.readlines() if w.strip()]
	return set(stopwords)

stopw = "stoplist_russian.txt"
print(load_stop_words(stopw))

{'этому', 'нам', 'будет', 'два', 'моего', 'того', 'вам', 'своем', 'всею', 'она', 'была', 'и', 'которой', 'моею', 'своя', 'моими', 'кому', 'этими', 'одна', 'таким', 'своему', 'было', 'нем', 'себя', 'можешь', 'моей', 'только', 'этим', 'может', 'быть', 'нею', 'кем', 'наша', 'так', 'моëм', 'могла', 'могли', 'ним', 'таких', 'эту', 'нашим', 'своих', 'вас', 'ее', 'одними', 'даже', 'ему', 'с', 'моим', 'тот', 'нас', 'своей', 'что-то', 'эта', 'его', 'мою', 'свои', 'пока', 'едва', 'наши', 'одного', 'всю', 'вами', 'одной', 'вы', 'были', 'будут', 'всеми', 'ей', 'тебя', 'моем', 'которому', 'всего', 'ещë', 'которым', 'ними', 'моему', 'такою', 'нашу', 'каждый', 'на', 'могут', 'тобой', 'уже', 'этом', 'такому', 'наших', 'главный', 'будь', 'свою', 'собою', 'когда', 'которые', 'я', 'мною', 'ней', 'тобою', 'который', 'мой', 'мое', 'той', 'мои', 'своё', 'в', 'могу', 'такое', 'всех', 'есть', 'моя', 'здесь', 'что', 'не', 'по', 'которого', 'него', 'чего', 'будем', 'еë', 'мной', 'себе', 'одно', 'ими', 'самый', 

### Препроцессинг, лемматизация

In [418]:
def preprocessing(raw_text):
    clean_text = re.sub('\W+', ' ', raw_text) # \W = [^a-zA-Z0-9_]
    return clean_text

def lemmatize(input):
    return [lemma.strip() for lemma in m.lemmatize(preprocessing(input.lower())) if lemma.strip()]

text = "с виду воду, которая бьёт  рядом с французским городом Авен"
m.lemmatize(text)

#r_text = "Он еле дождался, пока я доковыряюсь ключом в замке и, не снимая сапожек, прямиком пошёл в свою спальню."
print(preprocessing(text))
print(lemmatize(text))

с виду воду которая бьёт рядом с французским городом Авен
['с', 'вид', 'вода', 'который', 'бить', 'рядом', 'с', 'французский', 'город', 'авен']


In [419]:
def remove_stop_words(lemmas, stopwords):
    return ' '.join([word for word in lemmas if word not in stopwords])

#stop_words_filename = 'D:/GooDrive/0_2018_Edu_CL2/0_2018_Edu_HSE_Bak_CL2/WSD/WSD_Classifier/stoplist_russian.txt' 	# одно слово на строке
stop_words = load_stop_words(stopw)
raw_texts = ['Он', 'еле', 'дождался', 'пока', 'я', 'доковыряюсь', 'ключом', 'в', 'замке', 'и', 'не', 'снимая']
print(remove_stop_words(raw_texts, stop_words))

Он еле дождался доковыряюсь ключом замке снимая


### Найти наиболее частотные слова в корпусе (for reduced vectors)

In [459]:
import string
from collections import Counter

f1 = open('context_words.txt', 'r', encoding = 'utf-8')
text = f1.read()
text = text.strip('\n')

exclude = set(string.punctuation)
text = ''.join(ch for ch in text if ch not in exclude)
text = text.split(' ')

# remove stop-words
text = remove_stop_words(text, stop_words)

# lemmatize
text = lemmatize(text)

# counter
counts = Counter(text)
counts = sorted(counts.items(), key=lambda pair: pair[1], reverse=True)

# list of most popular words
popular = []

for i in counts:
    if i[1] > 10:
        popular.append(i[0])

print(len(popular))

205


### 1. Загрузка и адаптация корпуса (initial vectors)



In [503]:
import codecs

def remove_stop_words(lemmas, stopwords):
    return ' '.join([word for word in lemmas if word not in stopwords])

def load_files(filename):
    # список, где будут накапливаться результаты
    results = []
    
    # открываем файл как переменную inp_file.
    # Важно помнить, что в этой переменной хранится не сам текстовый файл, а своего рода указатель на него.
    # Файл еще предстоит прочитать.
    # Конструкция with следит за тем, чтобы переменная, которая после as, существовала только внутри блока.
    # Благодаря этому файл закроется сразу же после использования
    with(codecs.open(filename, encoding = 'utf-8')) as inp_file:
        # читаем первую строку "в никуда" (не сохраняем результат).
        # Первая строка у нас — это названия колонок, они нам не нужны
        inp_file.readline()
        # Для каждой строки в файле
        for line in inp_file:
            # откусываем концы строк
            line = line.strip('\r\n')
            # строка.count(подстрока) возвращает количество вхождений подстроки в строку
            if line.count('\t') != 1:
            # специальное ключевое слово для перехода к следующей итерации цикла
                continue
            # делим строку по табуляции и складываем в каждую из двух переменных очередное значение
            sence, context = line.split('\t')
            # добавляем кортеж (sence, context) в общий список результатов
            
            cx = lemmatize(context)
            #cx = context
            cx = remove_stop_words(cx, stop_words)
            results.append((int(sence), cx))
    # возвращаем результаты в виде списка кортежей (sence, context)
    return results

### 1. Загрузка и адаптация корпуса для уменьшенной размерности вектора (reduced vectors)

In [460]:
import codecs

def remove_stop_words(lemmas, stopwords): # also removes popular words
    return ' '.join([word for word in lemmas if word not in stopwords and word not in popular])

def load_files(filename):
    # список, где будут накапливаться результаты
    results = []
    
    with(codecs.open(filename, encoding = 'utf-8')) as inp_file:
        inp_file.readline()
        for line in inp_file:
            line = line.strip('\r\n')
            if line.count('\t') != 1:
                continue
            sence, context = line.split('\t')  
            cx = lemmatize(context)
            # remove most popular words + stop words
            cx = remove_stop_words(cx, stop_words)
                    
            results.append((int(sence), cx))

    return results

In [461]:
# указываем абсолютный или относительный путь
data = load_files('castle-lock.txt')

In [463]:
with codecs.open('lemmatized_data.txt', mode='w', encoding='utf-8') as wr:
    wr.write('sense\tcontext(lemmatized)\r\n')
    for item in data:
        wr.write(str(item[0]) + '\t' + item[1] + "\r\n")

1. Перевод корпуса в векторы признаков, feature vectors

In [464]:
# математика
import numpy

# разные классификаторы
from sklearn.naive_bayes import MultinomialNB       # Multinomial Naive Bayes
#from sklearn import tree
from sklearn.tree import DecisionTreeClassifier     # Decision Tree
from sklearn.ensemble import RandomForestClassifier # Random Forest
#from sklearn.svm import LinearSVC

# векторизация
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

# деление корпуса train/test
from sklearn.model_selection import cross_val_score # cross-validation
from sklearn.model_selection import train_test_split

# Pipeline
from sklearn.pipeline import Pipeline

In [465]:
x_context = [item[1] for item in data]
y_labels = [item[0] for item in data]
print(x_context[:5])
print(y_labels[1])

['горный склон конный спорт рыбалка круиз исторический пляжный', 'принцесса смело спасать распрекрасный уносить', 'сыграть роль огонь сказочный светлячок летний сад модный', 'семь чудо свет величественный святой петр xv сегодня', 'святой петр xv сегодня подводный археология']
1


In [466]:
def tokenize(input):
    return input.split(' ')

In [467]:
#classifiers = [('SVM', LinearSVC), ('Bayes', MultinomialNB)]
classifiers = [('Bayes', MultinomialNB), ('Decision Tree', DecisionTreeClassifier), ('Random Forest', RandomForestClassifier)]
vectorizers = [('Counts', CountVectorizer), ('TfIdf', TfidfVectorizer)]

### F-score for classifier

In [468]:
def score_classifier(clf, metric='f1'):
    scores = cross_val_score(pipeline, numpy.asarray(x_context), numpy.asarray(y_labels), cv=5, scoring=metric)
    score = sum(scores) / len(scores)    
    return score

### Pipeline score estimation

In [469]:
for clf in classifiers:
    for vctr in vectorizers:
        pipeline = Pipeline([
    ('vectorizer', vctr[1]()),
    ('classifier', clf[1]()) ])
        print (clf[0], vctr[0], score_classifier(pipeline))

Bayes Counts 0.820180351128
Bayes TfIdf 0.817795762006
Decision Tree Counts 0.531902289142
Decision Tree TfIdf 0.561793068912
Random Forest Counts 0.43316290704
Random Forest TfIdf 0.471462328458


In [471]:
#import sklearn

from nltk.corpus import stopwords
from sklearn.cross_validation import train_test_split
from sklearn.cross_validation import KFold
from sklearn.naive_bayes import MultinomialNB
from sklearn import tree
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC

from sklearn.metrics import precision_recall_curve
from sklearn.metrics import average_precision_score
from sklearn.model_selection import train_test_split

import numpy as np

#from sklearn.feature_extraction.text import BinaryVectorizer, CountVectorizer, TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

import os, sys, codecs

In [472]:
texts = [item[1] for item in data]
for i in range(10):
    print (texts[i])
    
len(texts)

print (3974 *16344 * 4)

горный склон конный спорт рыбалка круиз исторический пляжный
принцесса смело спасать распрекрасный уносить
сыграть роль огонь сказочный светлячок летний сад модный
семь чудо свет величественный святой петр xv сегодня
святой петр xv сегодня подводный археология
прибор вешать зал масштабный фамильный имение совместно
размер первоначальный выборгский основывать семь рано
информация товар конструктор рыцарский 2700 руб фантастический набор
возводить настоящий рыцарский упаковка придумывать
девичий 110 километр измир
259804224


### Vectorization: Tf idf

In [473]:
vectorizer = TfidfVectorizer() # Tf idf

texts = [item[1] for item in data]
vectorized = vectorizer.fit_transform(texts)
vectorized

print (vectorized[0], vectorized[0].shape)
print('\n')

names = vectorizer.get_feature_names()

  (0, 801)	0.314197591336
  (0, 3133)	0.367687688282
  (0, 1473)	0.367687688282
  (0, 3285)	0.367687688282
  (0, 3003)	0.367687688282
  (0, 1553)	0.367687688282
  (0, 1307)	0.300190700913
  (0, 2396)	0.367687688282 (1, 3978)




#### Classifier: Random Forest

In [474]:
clf = RandomForestClassifier(n_estimators=250)

In [475]:
y_labels = [item[0] for item in data]
X_train, X_test, y_train, y_test = train_test_split(vectorized, y_labels, test_size=0.1, random_state=42)

clf.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=250, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [476]:
items = clf.predict(X_test.toarray())
right = 0
for i in range(len(items)):
    if items[i] == y_test[i]:
        right += 1
        
print ('Total: %d\nRight: %d\nScore: %.3f' % (len(items), right, float(right) / len(items)))

words = {}
for i in range(len(clf.feature_importances_)):
    words[names[i]] = clf.feature_importances_[i]
    
for item in sorted(words, key = lambda word: words[word], reverse=True):
    print (item, words[item])

Total: 140
Right: 94
Score: 0.671
рыцарский 0.00625302102458
мир 0.0047283554258
королевский 0.00470000443424
небольшой 0.00469517829907
король 0.0046296230639
чешский 0.00456717378291
зал 0.0043742261472
принц 0.0042266778442
называть 0.00414630958502
легко 0.00390907214001
сигнализация 0.00389000407204
ремонт 0.00380373076587
надежность 0.00377722787808
английский 0.00373060508622
обеспечивать 0.00363437835186
европа 0.00362845981507
оснащать 0.00361551199949
японский 0.00361404158112
блокировка 0.00351418949161
проект 0.00343833504929
монастырь 0.00337578689399
остров 0.00332743199883
посещение 0.00328552551005
скала 0.00325422721659
теперь 0.00324236206142
европейский 0.00322254319927
воздушный 0.00310789834754
доступ 0.0030154922262
вскрывать 0.00294881028472
исторический 0.00294732289708
святой 0.00284230853789
долина 0.00277829700598
закрывать 0.00272912185232
памятник 0.00272458782861
повышенный 0.00264992930086
море 0.0026326220671
романтический 0.00258542306663
архитектурный 

град 0.000381007957523
18 0.000380998537718
постройка 0.000380173306846
свет 0.00038011259689
чаша 0.000379963408989
взять 0.000378817935615
вор 0.000378646173588
считать 0.000378199623375
подобный 0.000378117227012
отсиживаться 0.000376970539561
любимый 0.000376933106939
заповедник 0.000376427154131
модернизация 0.000375998090307
насыпь 0.000375673799142
англия 0.000374932016065
заменять 0.000373053227078
королева 0.000372698970944
устраиваться 0.000371770527086
содержать 0.000371358775209
вблизи 0.000370523188987
осуществлять 0.000370241765705
потеряться 0.000369194492871
двухсистемник 0.000368115982048
экологически 0.000367226566242
ремень 0.000365600611818
граф 0.000365502325286
встречаться 0.000365388971591
электропривод 0.000364970294714
ломать 0.000364104315
май 0.000362590343548
тупик 0.000362272787148
встроенный 0.000362224684583
шенбруннский 0.000360907398317
фестиваль 0.000360663583982
крево 0.000359478316022
нестандартный 0.000359284102387
высота 0.000358680843081
природный

мадаба 0.000126183711201
бронечашка 0.000126109103928
хитрость 0.000126023889951
часовня 0.000125939066425
драма 0.000125776922089
исключать 0.000125764380716
фирма 0.000125557679425
венецианский 0.000125538808349
прохождение 0.000125504884127
шотландия 0.000125497385777
продолжение 0.000125408026978
офицер 0.000125397026465
пастуший 0.000125292414307
суметь 0.000125283564224
альмаден 0.000125239047729
покупатель 0.00012522350255
оборона 0.000125086549416
поттер 0.000124839924883
разнообразный 0.000124754202798
рядом 0.00012473830666
доступный 0.000124576952164
сборка 0.000124550295093
рабочий 0.000124418341853
бес 0.00012436674053
решетчатый 0.00012432369279
скоро 0.00012423091431
красиво 0.00012422211593
момент 0.000124115964612
сообщать 0.000123799984765
близкий 0.000123757138758
четырехугольный 0.000123716806756
небесный 0.000123477135883
переулок 0.000123463997813
песок 0.000123406018681
алексей 0.000123370224473
фонтанка 0.000123355809393
иметься 0.000123129579248
свадебный 0.000

вв 6.51249320375e-05
пилниц 6.51131538778e-05
противодымный 6.50420670363e-05
троицкий 6.50400216113e-05
бокс 6.50081167758e-05
захлопывание 6.48864845117e-05
происхождение 6.48108641784e-05
слава 6.47600792148e-05
страждущий 6.47290726202e-05
каравансарай 6.46892485688e-05
кровь 6.46864068667e-05
новеченто 6.45302160073e-05
выдвигать 6.43969045636e-05
опять 6.43358905303e-05
сан 6.42893868894e-05
огораживать 6.41732996207e-05
тим 6.40919376178e-05
австрийский 6.40874851311e-05
включаться 6.4008287847e-05
управлять 6.4003306879e-05
золота 6.39723822358e-05
wat 6.37579951378e-05
трирема 6.36881133731e-05
дрезден 6.36876350333e-05
лист 6.35459717751e-05
прежний 6.34815358433e-05
телескопический 6.34352154068e-05
окончание 6.33720236739e-05
ледница 6.33442185901e-05
кирялаботный 6.31640362947e-05
промывать 6.30982163105e-05
напротив 6.30548555137e-05
немедленно 6.294224146e-05
пешком 6.2911720846e-05
ударно 6.28131537903e-05
приезжать 6.26456313212e-05
отпирание 6.25542686369e-05
сарай 6.

се 1.22859401127e-05
1994 1.22805997666e-05
смонтировать 1.22397512304e-05
пойти 1.22304465908e-05
1740 1.22249261484e-05
планироваться 1.22193489409e-05
аристократ 1.21366984184e-05
потеря 1.20984544494e-05
элегантно 1.20489967881e-05
обольщаться 1.19258748234e-05
копенгаген 1.19244216813e-05
неудобно 1.18556239069e-05
штехельберг 1.18292335695e-05
безопасный 1.17439776574e-05
фицхью 1.17133651559e-05
свойство 1.16761173172e-05
кочевский 1.16481733736e-05
учить 1.15948239496e-05
передняя 1.15808546652e-05
древнегреческий 1.15749433163e-05
поистине 1.15565712177e-05
здорово 1.15248172518e-05
вкл 1.15057410697e-05
тайна 1.14593544313e-05
теплоизоляция 1.14442330479e-05
стычка 1.14432024335e-05
лопата 1.14336615253e-05
квест 1.13287118435e-05
средиземноморье 1.13242815752e-05
полоса 1.1281147919e-05
ткм 1.12590956793e-05
кстати 1.12267571551e-05
венцовый 1.12083686742e-05
судоверфь 1.11918304407e-05
акцентировать 1.11735881926e-05
отмычка 1.11113759764e-05
компонент 1.11011091277e-05
цеп

#### Tf idf & Random forest f-score:

In [477]:
from sklearn.metrics import f1_score

f1_score(y_test, items, labels=None, pos_label=1, average='binary', sample_weight=None)

0.58181818181818179

#### Classifier: Multinomial naive Bayes

In [478]:
clf = MultinomialNB()

In [479]:
y_labels = [item[0] for item in data]
X_train, X_test, y_train, y_test = train_test_split(vectorized, y_labels, test_size=0.1, random_state=42)
clf.fit(X_train, y_train)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [480]:
items = clf.predict(X_test.toarray())
right = 0
for i in range(len(items)):
    if items[i] == y_test[i]:
        right += 1
        
print ('Total: %d\nRight: %d\nScore: %.3f' % (len(items), right, float(right) / len(items)))

#words = {}
#for i in range(len(clf.feature_importances_)):
#    words[names[i]] = clf.feature_importances_[i]     # no feature_importances_ in MultinomialNB
    
for item in sorted(words, key = lambda word: words[word], reverse=True):
    print (item, words[item])

Total: 140
Right: 119
Score: 0.850
рыцарский 0.00625302102458
мир 0.0047283554258
королевский 0.00470000443424
небольшой 0.00469517829907
король 0.0046296230639
чешский 0.00456717378291
зал 0.0043742261472
принц 0.0042266778442
называть 0.00414630958502
легко 0.00390907214001
сигнализация 0.00389000407204
ремонт 0.00380373076587
надежность 0.00377722787808
английский 0.00373060508622
обеспечивать 0.00363437835186
европа 0.00362845981507
оснащать 0.00361551199949
японский 0.00361404158112
блокировка 0.00351418949161
проект 0.00343833504929
монастырь 0.00337578689399
остров 0.00332743199883
посещение 0.00328552551005
скала 0.00325422721659
теперь 0.00324236206142
европейский 0.00322254319927
воздушный 0.00310789834754
доступ 0.0030154922262
вскрывать 0.00294881028472
исторический 0.00294732289708
святой 0.00284230853789
долина 0.00277829700598
закрывать 0.00272912185232
памятник 0.00272458782861
повышенный 0.00264992930086
море 0.0026326220671
романтический 0.00258542306663
архитектурный

некоторый 0.000463557905038
одноименный 0.000462816788719
вот 0.000462211805055
бронезащита 0.000461747819967
практически 0.000461732726545
требовать 0.00045930598007
токугава 0.000459296159296
уязвимый 0.000458627429456
укрепление 0.000458615410407
ведь 0.000458223598158
многоуровневый 0.000457890225999
приходиться 0.000456551639522
свободный 0.000454861712219
атмосфера 0.000454748551322
картина 0.00045427325051
коробка 0.000453409153708
традиция 0.000453291527904
первичный 0.000452849455649
аббас 0.000452069811228
окружать 0.000451181877968
повышать 0.00045090766802
зваться 0.000450595308252
мало 0.000450361800401
колдунья 0.000450113323387
12 0.000447545917161
снежный 0.000447290919157
внимание 0.000447081306347
belimo 0.000446922084782
близ 0.0004466029006
служить 0.000446303119107
вавельский 0.000445589958827
менее 0.000445290588311
катиться 0.000442976746784
поворот 0.000441906926359
семья 0.000441780182893
черный 0.000439053245752
особенно 0.00043863766777
стапятидесятисемилетни

холодильник 0.000104972535645
парктроник 0.000104846476683
town 0.000104754683785
восприятие 0.000104711711364
пролегать 0.000104669425043
японец 0.000104644933123
нагреватель 0.000104535201065
вероятно 0.000104512337522
припасы 0.00010447603965
комфорт 0.000104252983545
гессенский 0.000104224593608
сечение 0.00010410800106
цапля 0.000104080264795
электростеклоподъемник 0.000104064144074
соколенок 0.00010400952138
масштаб 0.000103829746561
однозначно 0.00010366460656
оск 0.000103624410357
15 0.000103613597006
лязг 0.000103583004236
trsat 0.000103559926106
усыпать 0.000103325005852
копать 0.000103275390639
контекст 0.00010327018164
марвау 0.000103267696687
казаться 0.000103209186332
враждебный 0.00010297857442
торжество 0.000102788492077
доводиться 0.000102712412737
пора 0.000102660307996
останавливаться 0.000102654510036
ой 0.000102642411089
конви 0.00010263580168
карпатский 0.000102333270077
половина 0.000102267189963
изготавливаться 0.000102261586416
отважиться 0.000102188741478
суще

овощная 2.03719408727e-05
бомбить 2.02893252419e-05
пожить 2.02874736269e-05
фейсбук 2.02835652854e-05
жизненный 2.02810289049e-05
никуда 2.02493635778e-05
среда 2.02383484566e-05
затруднение 2.02166947271e-05
крюк 2.01961226077e-05
уе 2.00964171486e-05
приваривать 2.00952058459e-05
тюрьма 2.00725076764e-05
царь 1.9975643508e-05
кэстлы 1.97493060726e-05
телекоммуникация 1.97114743754e-05
1715 1.97092368882e-05
кампания 1.97073315445e-05
светский 1.96176328498e-05
сноу 1.95661049995e-05
эриктип 1.95308935964e-05
клавиша 1.94783367767e-05
динамичный 1.94308849507e-05
обтягивать 1.93934070434e-05
клониться 1.93459772732e-05
храповой 1.93371012414e-05
kale 1.9325916056e-05
сравнение 1.92841448311e-05
ребро 1.92481892081e-05
сосватывать 1.90786106228e-05
истинный 1.90712126396e-05
отрывание 1.89735208974e-05
валалт 1.89341177339e-05
телефон 1.89199944833e-05
стройка 1.8911003149e-05
роман 1.89025510289e-05
чистый 1.88321927793e-05
забота 1.8772261885e-05
срочно 1.86291461534e-05
клиент 1.86

#### Tf idf & multinomial naive Bayes f-score:

In [482]:
f1_score(y_test, items, labels=None, pos_label=1, average='binary', sample_weight=None)

0.8571428571428571

#### Classifier: Decision Tree

In [483]:
clf = tree.DecisionTreeClassifier()

In [484]:
y_labels = [item[0] for item in data]
X_train, X_test, y_train, y_test = train_test_split(vectorized, y_labels, test_size=0.1, random_state=42)
clf.fit(X_train, y_train)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

In [485]:
items = clf.predict(X_test.toarray())
right = 0
for i in range(len(items)):
    if items[i] == y_test[i]:
        right += 1
        
print ('Total: %d\nRight: %d\nScore: %.3f' % (len(items), right, float(right) / len(items)))

words = {}
for i in range(len(clf.feature_importances_)):
    words[names[i]] = clf.feature_importances_[i]
    
for item in sorted(words, key = lambda word: words[word], reverse=True):
    print (item, words[item])

Total: 140
Right: 98
Score: 0.700
чешский 0.00842008109907
мир 0.00828535980149
интересный 0.00819322638044
монастырь 0.00816888727963
рыцарский 0.00815384615385
называть 0.00804665953979
японский 0.00792715469514
небольшой 0.00781029246376
королевский 0.00769599550087
теперь 0.00746693007032
король 0.00736611076135
долина 0.00693675030404
скала 0.00685088425253
английский 0.00676660272394
посещение 0.00668386697011
надежность 0.00665710352457
исторический 0.00660263942013
принц 0.00652288363766
зал 0.00635292193228
проект 0.00628317347557
xiv 0.00621456739577
воздушный 0.00614707888106
море 0.00608068378964
сад 0.00601535862806
кощей 0.00588086871771
предел 0.00582410357565
остров 0.00576815637511
здание 0.00571301147669
северный 0.00565865361297
лес 0.005605067878
хотя 0.00555223971704
петр 0.00550015491669
назад 0.00545011682186
романтический 0.00544879959534
архитектура 0.00540455680663
ребенок 0.0053981601939
допы 0.00535956569898
европа 0.00534822346685
устраивать 0.0053151340662

восприятие 0.0
восток 0.0
восторг 0.0
восточный 0.0
востребовать 0.0
восхищение 0.0
вот 0.0
впервые 0.0
впечатление 0.0
впечатляющий 0.0
впитывать 0.0
вполне 0.0
враг 0.0
враждебный 0.0
врата 0.0
врач 0.0
врезать 0.0
врезаться 0.0
врубка 0.0
вс 0.0
всевозможный 0.0
всемирный 0.0
всерьез 0.0
вскрывать 0.0
вскрытие 0.0
вслед 0.0
вспоминать 0.0
вспомогательный 0.0
вставать 0.0
вставка 0.0
вставлять 0.0
встраивать 0.0
встреча 0.0
встречать 0.0
встречаться 0.0
встроенный 0.0
вступать 0.0
всякий 0.0
вторгаться 0.0
вторичный 0.0
втроем 0.0
вх 0.0
вход 0.0
въезд 0.0
выбивание 0.0
выбивать 0.0
выборг 0.0
выборы 0.0
вывесить 0.0
вывод 0.0
выводить 0.0
вывозить 0.0
выгодно 0.0
выгодный 0.0
выгозеро 0.0
выдавать 0.0
выдающийся 0.0
выдвигать 0.0
выдвигаться 0.0
выдвижной 0.0
выдерживать 0.0
выдубич 0.0
выезд 0.0
выездной 0.0
выезжать 0.0
выживать 0.0
вызов 0.0
вызывать 0.0
выключать 0.0
выламывать 0.0
вылет 0.0
вынимание 0.0
выниматься 0.0
вынос 0.0
вынуждать 0.0
выпадать 0.0
выпивать 0.0
выполнять

мало 0.0
малый 0.0
малыш 0.0
мальдивы 0.0
мальчик 0.0
маляр 0.0
манер 0.0
маннсфильд 0.0
марвау 0.0
марганцовистый 0.0
марианский 0.0
марка 0.0
маркат 0.0
марксбург 0.0
мартин 0.0
марухубить 0.0
марципановый 0.0
маршрут 0.0
маска 0.0
масса 0.0
массив 0.0
массивный 0.0
массовый 0.0
мастер 0.0
масштаб 0.0
масштабный 0.0
математически 0.0
материал 0.0
мать 0.0
мацоха 0.0
мачеха 0.0
машина 0.0
мгновенный 0.0
мда 0.0
мдф 0.0
мебель 0.0
меблировка 0.0
мегаполис 0.0
медовый 0.0
международный 0.0
межкомнатный 0.0
межшкольный 0.0
мелкий 0.0
мельница 0.0
менее 0.0
меньшиковский 0.0
менять 0.0
меняться 0.0
мера 0.0
мероприятие 0.0
местечко 0.0
мести 0.0
местность 0.0
местный 0.0
местоположение 0.0
месторасположение 0.0
месяц 0.0
металлитюот 0.0
металлургия 0.0
метод 0.0
метр 0.0
меч 0.0
мечеть 0.0
мечта 0.0
мечтать 0.0
мешать 0.0
микродент 0.0
микропереключатель 0.0
миллион 0.0
миллионер 0.0
миля 0.0
мимо 0.0
мина 0.0
миндовга 0.0
мини 0.0
минимальный 0.0
минимум 0.0
мирно 0.0
мировой 0.0
мистиче

пружина 0.0
пружинный 0.0
прыгать 0.0
прынц 0.0
прямо 0.0
прямоугольный 0.0
птица 0.0
пу 0.0
публика 0.0
пуболь 0.0
пул 0.0
пулава 0.0
пунква 0.0
пункевня 0.0
пункт 0.0
пускай 0.0
пускать 0.0
пустошь 0.0
пусть 0.0
путем 0.0
путешественник 0.0
путешествие 0.0
пушка 0.0
пытаться 0.0
пытка 0.0
пышный 0.0
пьеса 0.0
пятиклассник 0.0
пятый 0.0
пять 0.0
раб 0.0
рабад 0.0
работник 0.0
рабочий 0.0
радиоуправление 0.0
радовать 0.0
радоваться 0.0
разбирать 0.0
разбираться 0.0
разбойник 0.0
разборка 0.0
развалина 0.0
развивать 0.0
развитие 0.0
развлекательный 0.0
разворачиваться 0.0
разгадывать 0.0
разгонять 0.0
раздельный 0.0
разламывать 0.0
разлучаться 0.0
размер 0.0
размещать 0.0
размещение 0.0
разница 0.0
разновидность 0.0
разнообразие 0.0
разнообразный 0.0
разработка 0.0
разрастаться 0.0
разрешать 0.0
разрушение 0.0
разрывать 0.0
разряд 0.0
разряжать 0.0
разумеется 0.0
разумный 0.0
ракля 0.0
рама 0.0
рамбуйя 0.0
ранее 0.0
ранний 0.0
рано 0.0
рапунцель 0.0
раса 0.0
раскрываться 0.0
раскрытие 0

#### Tf idf & Decision Tree f-score:

In [487]:
f1_score(y_test, items, labels=None, pos_label=1, average='binary', sample_weight=None)

0.66129032258064513

### Vectorization: Count 

In [488]:
vectorizer = CountVectorizer() # Count

texts = [item[1] for item in data]
vectorized = vectorizer.fit_transform(texts)
vectorized

print (vectorized[0], vectorized[0].shape)

  (0, 2396)	1
  (0, 1307)	1
  (0, 1553)	1
  (0, 3003)	1
  (0, 3285)	1
  (0, 1473)	1
  (0, 3133)	1
  (0, 801)	1 (1, 3978)


#### Classifier: Random Forest

In [489]:
clf = RandomForestClassifier(n_estimators=250)

In [490]:
y_labels = [item[0] for item in data]
X_train, X_test, y_train, y_test = train_test_split(vectorized, y_labels, test_size=0.1, random_state=42)
clf.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=250, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [491]:
items = clf.predict(X_test.toarray())
right = 0
for i in range(len(items)):
    if items[i] == y_test[i]:
        right += 1
    #print items[i], y_test[i]
names = vectorizer.get_feature_names()
words = {}
print ('Total: %d\nRight: %d\nScore: %.3f' % (len(items), right, float(right) / len(items)))
for i in range(len(clf.feature_importances_)):
    words[names[i]] = clf.feature_importances_[i]
    
for item in sorted(words, key = lambda word: words[word], reverse=True):
    print (item, words[item])

Total: 140
Right: 95
Score: 0.679
рыцарский 0.0065607588484
мир 0.00544595617712
небольшой 0.00525637812862
королевский 0.00509165345934
король 0.00497602722006
зал 0.00462935189408
чешский 0.00448976547911
называть 0.0044221583789
принц 0.00433388092073
блокировка 0.00427095256161
английский 0.0041671765348
проект 0.00396413718199
европа 0.00386159597963
посещение 0.00377634500707
европейский 0.00372952187219
надежность 0.00365801190344
японский 0.00363779585304
легко 0.00357634175777
сигнализация 0.00353578351754
остров 0.00349051931702
скала 0.00348780962608
ремонт 0.00347967046638
теперь 0.00344453298038
монастырь 0.00340254242396
воздушный 0.00330900309202
исторический 0.00327713826277
обеспечивать 0.00316988918592
романтический 0.00301635025227
долина 0.00294078382713
памятник 0.00292478680199
святой 0.00289870130558
предел 0.00289670956814
здание 0.00286381948107
вскрывать 0.00276996061877
доступ 0.00276287595941
повышенный 0.00273831643684
море 0.00272795183073
оснащать 0.00268

ландшафт 0.00044170149507
мышь 0.000441024587999
заменять 0.000439633620222
захватить 0.000439314150313
взять 0.000439018956241
отсиживаться 0.000438299583562
броненакладка 0.000436412139072
играть 0.000435932936219
град 0.000435314995052
двухсистемник 0.000435122757454
развлекательный 0.000434895634941
комедия 0.000434015686627
содержать 0.000433873837077
обороноспособность 0.000433652491628
белый 0.000432964661435
корабль 0.000432708168751
снова 0.000432410344816
замечать 0.000432320837685
двенадцатый 0.000430386106643
попытка 0.000429694848658
круг 0.00042889065817
наряду 0.000428474195913
городище 0.00042840635419
сморгонское 0.000427998810236
рассказывать 0.000426905543949
взламывать 0.000426505205601
мощность 0.000424780152483
хмельник 0.000422385514639
бревно 0.000422300489398
романтика 0.000421467536178
абиса 0.000420658468094
восхитительный 0.000420233704148
ущерб 0.0004173884533
румыния 0.000417361297625
написать 0.000417141030274
причина 0.000416278916364
девичий 0.000415349

искусство 0.000167688557534
нова 0.000167550008233
туристический 0.000167340706695
графиня 0.000167192357084
стадия 0.000166870986032
онлайн 0.000166808664699
тушеный 0.00016643587373
панис 0.000166397734222
тарсатый 0.000166296544871
сечение 0.000166265006194
баланс 0.000166070061682
крат 0.000166023625186
применяться 0.000165761891614
признак 0.000165687966367
открытие 0.000165680742265
удаваться 0.000165671437766
многообразный 0.000165666227074
ni 0.000165589769242
число 0.000165529483842
разлучаться 0.000165485096794
сундук 0.000165395175846
держать 0.000164995702368
солнце 0.000164995526246
многое 0.000164561093384
темно 0.00016456004273
нести 0.000164491210572
замолвить 0.000164160451713
андергрунда 0.000164085860964
настолько 0.000163788123483
сохранность 0.000163395233969
подниматься 0.000163245644372
накупаться 0.000162905202278
доезжать 0.000162870189993
поездка 0.000162861111023
вслед 0.000162857960745
связанный 0.000162668178819
крит 0.000162558042102
новослободский 0.00016

заводить 8.49712118997e-05
влиять 8.4962714505e-05
силуэт 8.49428440504e-05
обряд 8.48811334947e-05
некоторые 8.48554813282e-05
золота 8.48289950762e-05
даниэль 8.46873352289e-05
изначально 8.46600716334e-05
крестьянский 8.45121649726e-05
супер 8.446786773e-05
гнев 8.43961991301e-05
сделать 8.43266320515e-05
покидать 8.42712493464e-05
белка 8.42710710262e-05
фонтанка 8.42630024612e-05
перепад 8.4029540662e-05
турьякский 8.39681533178e-05
гитлер 8.38606045738e-05
убеждаться 8.38016945045e-05
швангау 8.37584941167e-05
гегель 8.37550765363e-05
древесный 8.29692973746e-05
ответственность 8.29601787341e-05
разрастаться 8.2948943179e-05
поблизости 8.29077225503e-05
океан 8.28171086738e-05
км 8.26981978909e-05
дизайн 8.26627230831e-05
сосна 8.26504401017e-05
работник 8.25542148392e-05
подобранный 8.25317176437e-05
рождаться 8.2507196837e-05
хрупкий 8.24806193574e-05
дождаться 8.23075868832e-05
размещение 8.23060568817e-05
посейдон 8.22985978636e-05
тюремный 8.21530309787e-05
терраса 8.2020273

фридрихштейн 2.24634644756e-05
заиорданья 2.24355132761e-05
14 2.24274557327e-05
hо 2.23579204298e-05
аварийный 2.22805363507e-05
потеря 2.21096789037e-05
грабитель 2.19551091308e-05
мишель 2.19362517741e-05
укреплять 2.18181274472e-05
фол 2.17444868158e-05
маркат 2.16770323313e-05
интернет 2.16619082466e-05
сумерки 2.16563406631e-05
рассказ 2.16339838706e-05
дождь 2.1611237837e-05
навстречу 2.15480656864e-05
предложение 2.15055256089e-05
древнегреческий 2.14778654759e-05
происхождение 2.14771442601e-05
кло 2.14367561384e-05
амбуаз 2.1413579769e-05
автобус 2.1386637836e-05
оцилиндровать 2.13682143843e-05
стеклоподъемник 2.13187073491e-05
полотенце 2.12623993668e-05
вероятный 2.12601077319e-05
авиакомпания 2.12559621157e-05
23 2.12463250117e-05
деревня 2.12232527069e-05
деятель 2.1199781172e-05
тратить 2.11033860087e-05
маляр 2.10831988151e-05
экономика 2.10573693209e-05
глубина 2.10560070285e-05
кэстлы 2.1050950027e-05
строевой 2.10491541756e-05
ухмыляться 2.09900571489e-05
появление 2

#### Count & Random Forest f-score:

In [493]:
f1_score(y_test, items, labels=None, pos_label=1, average='binary', sample_weight=None)

0.59459459459459463

#### Classifier: Multinomial naive Bayes

In [494]:
clf = MultinomialNB()

In [495]:
y_labels = [item[0] for item in data]
X_train, X_test, y_train, y_test = train_test_split(vectorized, y_labels, test_size=0.1, random_state=42)
clf.fit(X_train, y_train)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [496]:
items = clf.predict(X_test.toarray())
right = 0
for i in range(len(items)):
    if items[i] == y_test[i]:
        right += 1
        
print ('Total: %d\nRight: %d\nScore: %.3f' % (len(items), right, float(right) / len(items)))

#words = {}
#for i in range(len(clf.feature_importances_)):
#    words[names[i]] = clf.feature_importances_[i]     # no feature_importances_ in MultinomialNB
    
for item in sorted(words, key = lambda word: words[word], reverse=True):
    print (item, words[item])

Total: 140
Right: 119
Score: 0.850
рыцарский 0.0065607588484
мир 0.00544595617712
небольшой 0.00525637812862
королевский 0.00509165345934
король 0.00497602722006
зал 0.00462935189408
чешский 0.00448976547911
называть 0.0044221583789
принц 0.00433388092073
блокировка 0.00427095256161
английский 0.0041671765348
проект 0.00396413718199
европа 0.00386159597963
посещение 0.00377634500707
европейский 0.00372952187219
надежность 0.00365801190344
японский 0.00363779585304
легко 0.00357634175777
сигнализация 0.00353578351754
остров 0.00349051931702
скала 0.00348780962608
ремонт 0.00347967046638
теперь 0.00344453298038
монастырь 0.00340254242396
воздушный 0.00330900309202
исторический 0.00327713826277
обеспечивать 0.00316988918592
романтический 0.00301635025227
долина 0.00294078382713
памятник 0.00292478680199
святой 0.00289870130558
предел 0.00289670956814
здание 0.00286381948107
вскрывать 0.00276996061877
доступ 0.00276287595941
повышенный 0.00273831643684
море 0.00272795183073
оснащать 0.0026

деревушка 0.000447658968777
услуга 0.000447262949531
кхо 0.000445405314972
тяга 0.000445289591567
дешевый 0.00044320901396
запись 0.000443072911312
ландшафт 0.00044170149507
мышь 0.000441024587999
заменять 0.000439633620222
захватить 0.000439314150313
взять 0.000439018956241
отсиживаться 0.000438299583562
броненакладка 0.000436412139072
играть 0.000435932936219
град 0.000435314995052
двухсистемник 0.000435122757454
развлекательный 0.000434895634941
комедия 0.000434015686627
содержать 0.000433873837077
обороноспособность 0.000433652491628
белый 0.000432964661435
корабль 0.000432708168751
снова 0.000432410344816
замечать 0.000432320837685
двенадцатый 0.000430386106643
попытка 0.000429694848658
круг 0.00042889065817
наряду 0.000428474195913
городище 0.00042840635419
сморгонское 0.000427998810236
рассказывать 0.000426905543949
взламывать 0.000426505205601
мощность 0.000424780152483
хмельник 0.000422385514639
бревно 0.000422300489398
романтика 0.000421467536178
абиса 0.000420658468094
восхи

бернадотт 0.000218671376281
конюх 0.000218510207227
выправка 0.000218390057229
массивный 0.000218126535412
писатель 0.000218120130817
легальный 0.00021787455371
злоумышленник 0.000217527505756
джигит 0.000217379764766
видно 0.000217191707092
нет 0.000217067503049
копеник 0.000217012402422
крутой 0.000216699044581
зубоврачебный 0.000216356576665
45 0.000216214962646
причудливый 0.00021544337344
машина 0.000215440236285
наймит 0.000215352644614
облицовка 0.000215342919199
св 0.000215157809692
перебивать 0.000215050629744
ну 0.000214987730867
коврик 0.0002148641747
ral 0.000214360546589
продукция 0.000214316082274
чума 0.000214095117258
перемещаться 0.00021402584514
догадка 0.000213999278697
фундамент 0.000213874736067
садить 0.000213830425166
осуществление 0.000213453045486
требование 0.000213423876251
stop 0.000212504101491
чудесный 0.000212447945799
бабки 0.000212114875438
театральный 0.000211898578106
задвижка 0.000211868851911
состояться 0.000211855354265
план 0.000211620150669
мыс 0

забор 8.82701769496e-05
скотч 8.82489521438e-05
лепта 8.81328924398e-05
лабиринт 8.8022904944e-05
обращать 8.78668679022e-05
исключать 8.78343359322e-05
бронечашка 8.77347662064e-05
модуль 8.7714822952e-05
деревенский 8.77004549037e-05
пес 8.767351304e-05
бомбить 8.75381836266e-05
мести 8.72366647821e-05
тм 8.72287985185e-05
теплообменник 8.71885462289e-05
вечер 8.71696362289e-05
документ 8.7062707208e-05
артемида 8.70070997654e-05
крепеж 8.67209870875e-05
гвард 8.65970750087e-05
ум 8.65145691533e-05
включаться 8.64351850445e-05
многосторонний 8.59328372232e-05
сено 8.58890919578e-05
диснейленд 8.57436503846e-05
canter 8.54605182279e-05
многозасовный 8.5355460383e-05
поиск 8.52572154713e-05
заводить 8.49712118997e-05
влиять 8.4962714505e-05
силуэт 8.49428440504e-05
обряд 8.48811334947e-05
некоторые 8.48554813282e-05
золота 8.48289950762e-05
даниэль 8.46873352289e-05
изначально 8.46600716334e-05
крестьянский 8.45121649726e-05
супер 8.446786773e-05
гнев 8.43961991301e-05
сделать 8.432663

продолжительность 5.2950272343e-05
фицхью 5.29489028391e-05
салат 5.28685992454e-05
82 5.28093230832e-05
варьироваться 5.26910082812e-05
опекун 5.25745483938e-05
аркадий 5.2403061251e-05
вдоль 5.22608627073e-05
сущность 5.21278015518e-05
1519 5.20739018842e-05
склонный 5.20483973285e-05
край 5.19699638167e-05
нида 5.18994399672e-05
переживать 5.18193479413e-05
правительственный 5.17830141987e-05
таять 5.17641205026e-05
ударно 5.17239873389e-05
разумный 5.17162004323e-05
mult 5.16454272693e-05
1926 5.16183303714e-05
киренский 5.15776038246e-05
нечаянно 5.15390628512e-05
ледница 5.15121638436e-05
разрешать 5.14932861412e-05
спас 5.14521703847e-05
труд 5.12531527615e-05
колумб 5.12258156086e-05
выниматься 5.11965260472e-05
знание 5.11697789539e-05
паперть 5.11574957639e-05
выпячивать 5.11543034447e-05
эмоция 5.10356239913e-05
пора 5.10330490128e-05
рубить 5.10112596484e-05
нота 5.0831792345e-05
внизу 5.06746121727e-05
царапаться 5.0595251289e-05
выпадать 5.0539209561e-05
корона 5.05071999

освещение 9.51589209367e-06
синан 9.48277963202e-06
довольный 9.44025845242e-06
аттачмен 9.42173642551e-06
цепь 9.4018644207e-06
1935 9.39157205743e-06
композит 9.38385749124e-06
поочередно 9.36904092282e-06
всякий 9.29790440617e-06
уютно 9.28586665361e-06
автозапуск 9.26232899178e-06
мировой 9.19694873053e-06
зарастать 9.18915573574e-06
тамбурный 9.10060567966e-06
шинирующий 9.00388235588e-06
перевод 8.970614061e-06
прожектор 8.85823444983e-06
карман 8.83815919103e-06
притягивать 8.83521420393e-06
вывозить 8.74282271685e-06
договор 8.7356690213e-06
бумага 8.69498493091e-06
ручной 8.68267500659e-06
собираться 8.65201638681e-06
обхват 8.56637466361e-06
культурно 8.56077676416e-06
220в 8.55725409746e-06
сжимать 8.55498000441e-06
сятихоко 8.51853694034e-06
глазомер 8.505717326e-06
лутка 8.47451417564e-06
57675 8.47425479537e-06
соединение 8.47099754842e-06
kale 8.46979107849e-06
комплектовать 8.43061093674e-06
выгозеро 8.35109208567e-06
фанера 8.34888208929e-06
эффективный 8.30795073578e-

#### Count & Multinomial naive Bayes f-score:

In [498]:
f1_score(y_test, items, labels=None, pos_label=1, average='binary', sample_weight=None)

0.8571428571428571

#### Classifier: Decision Tree

In [499]:
clf = tree.DecisionTreeClassifier()

In [500]:
y_labels = [item[0] for item in data]
X_train, X_test, y_train, y_test = train_test_split(vectorized, y_labels, test_size=0.1, random_state=42)
clf.fit(X_train, y_train)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

In [501]:
items = clf.predict(X_test.toarray())
right = 0
for i in range(len(items)):
    if items[i] == y_test[i]:
        right += 1
        
print ('Total: %d\nRight: %d\nScore: %.3f' % (len(items), right, float(right) / len(items)))

words = {}
for i in range(len(clf.feature_importances_)):
    words[names[i]] = clf.feature_importances_[i]
    
for item in sorted(words, key = lambda word: words[word], reverse=True):
    print (item, words[item])

Total: 140
Right: 106
Score: 0.757
рыцарский 0.00842008109907
чешский 0.00828535980149
небольшой 0.00816888727963
мир 0.00815384615385
монастырь 0.00804665953979
королевский 0.00792715469514
японский 0.00781029246376
называть 0.00769599550087
теперь 0.00746693007032
король 0.00736611076135
принц 0.00693675030404
xiv 0.00685088425253
исторический 0.00676660272394
английский 0.00668386697011
обеспечивать 0.00665710352457
посещение 0.00660263942013
остров 0.00652288363766
воздушный 0.00642383827013
проект 0.00635292193228
зал 0.00628317347557
романтический 0.00621456739577
сад 0.00614707888106
море 0.00608068378964
скала 0.00601535862806
лес 0.0058353906616
предел 0.00577928113601
северный 0.00572397701031
владелец 0.00566946294354
европа 0.0056157239583
кощей 0.00556274543039
река 0.00551051307893
святой 0.0054590129567
ребенок 0.00540823144082
архитектура 0.00528327433157
красивый 0.00523992438833
устраивать 0.00519710580293
обитатель 0.00515480992661
десяток 0.00511302828587
назад 0.00

безусловно 0.0
безуспешно 0.0
белка 0.0
белый 0.0
бенуа 0.0
бернадотт 0.0
бес 0.0
бескомпромиссный 0.0
бесплатно 0.0
бесплатный 0.0
бессмертный 0.0
бессмысленно 0.0
библиотека 0.0
библия 0.0
бизнес 0.0
биконс 0.0
битва 0.0
бл 0.0
благодаря 0.0
блеск 0.0
ближний 0.0
близ 0.0
близкий 0.0
близко 0.0
близнец 0.0
блог 0.0
блокиратор 0.0
блокировать 0.0
блокироваться 0.0
блокировка 0.0
блэк 0.0
бобкет 0.0
бог 0.0
богато 0.0
богатый 0.0
богеншперк 0.0
боготворить 0.0
бой 0.0
боковой 0.0
бокс 0.0
болгарка 0.0
болт 0.0
больно 0.0
больной 0.0
большинство 0.0
бомба 0.0
бомбить 0.0
боровать 0.0
борода 0.0
бороться 0.0
боумарис 0.0
бочка 0.0
боярин 0.0
бояться 0.0
брага 0.0
брадобрей 0.0
брас 0.0
брат 0.0
братчиков 0.0
брать 0.0
браться 0.0
бревен 0.0
бревенчатый 0.0
бревно 0.0
брелок 0.0
бренд 0.0
брестский 0.0
бретань 0.0
брешианский 0.0
брин 0.0
британский 0.0
бронезащита 0.0
броненакладка 0.0
бронечашка 0.0
бронинакладка 0.0
бронированный 0.0
броня 0.0
бросать 0.0
брус 0.0
брюква 0.0
брючный 0.

кваритир 0.0
кварнер 0.0
квартирный 0.0
квест 0.0
квеста 0.0
кегли 0.0
кедр 0.0
кейстер 0.0
кейт 0.0
кельн 0.0
кемпинг 0.0
кентерберийский 0.0
кеплер 0.0
керамонаполненный 0.0
керберос 0.0
кефалос 0.0
киото 0.0
кир 0.0
киренский 0.0
кирпич 0.0
кирялаботно 0.0
кирялаботный 0.0
кисловодск 0.0
клавиша 0.0
кладовщик 0.0
клан 0.0
клапан 0.0
класс 0.0
классицизм 0.0
клеить 0.0
клей 0.0
клиент 0.0
климат 0.0
клиновый 0.0
кло 0.0
клониться 0.0
клуб 0.0
ключевой 0.0
ключник 0.0
км 0.0
книга 0.0
кнопка 0.0
кнут 0.0
князь 0.0
ко 0.0
кобленц 0.0
кованый 0.0
коварство 0.0
ковбой 0.0
коверсад 0.0
коврик 0.0
кодак 0.0
кодирование 0.0
кожух 0.0
козел 0.0
козни 0.0
кол 0.0
колдунья 0.0
колесный 0.0
колесо 0.0
колесцовый 0.0
количество 0.0
коллекционер 0.0
коллекция 0.0
колодец 0.0
коломбьер 0.0
колония 0.0
колонна 0.0
колосс 0.0
колумб 0.0
колыхаться 0.0
кольцо 0.0
команда 0.0
командование 0.0
комбинация 0.0
комбинированный 0.0
комбинировать 0.0
комедия 0.0
комендант 0.0
комната 0.0
комплекс 0.0
компле

руб 0.0
рубенс 0.0
рубить 0.0
рубленый 0.0
рука 0.0
рулонный 0.0
руль 0.0
румыния 0.0
руно 0.0
русалочка 0.0
русский 0.0
русь 0.0
ручной 0.0
рыбалка 0.0
рывок 0.0
рыжий 0.0
рынок 0.0
рыночный 0.0
рыть 0.0
ряд 0.0
рядом 0.0
сага 0.0
сагрешский 0.0
садить 0.0
садовый 0.0
сажать 0.0
сайт 0.0
саксонский 0.0
салат 0.0
салон 0.0
салоу 0.0
сальвадор 0.0
сальватрьерра 0.0
самостраховка 0.0
самоцвет 0.0
самурай 0.0
сан 0.0
санкт 0.0
санто 0.0
сарай 0.0
сахар 0.0
сберегать 0.0
сбивать 0.0
сборка 0.0
св 0.0
свадебный 0.0
свадьба 0.0
сверление 0.0
сверло 0.0
сверхпрочный 0.0
сверху 0.0
свет 0.0
светло 0.0
светлый 0.0
светлячок 0.0
светодиод 0.0
светский 0.0
свободный 0.0
сводить 0.0
своеобразный 0.0
свойство 0.0
связанный 0.0
связывать 0.0
сгу 0.0
сделать 0.0
се 0.0
север 0.0
северо 0.0
сегодня 0.0
сегодняшний 0.0
сей 0.0
сейфовый 0.0
сейчас 0.0
секрет 0.0
секретка 0.0
секстеный 0.0
секция 0.0
сельский 0.0
сельхозпредприятие 0.0
семейство 0.0
семнадцать 0.0
семь 0.0
семья 0.0
сена 0.0
сено 0.0
сер

#### Count & Decision Tree f-score:

In [502]:
f1_score(y_test, items, labels=None, pos_label=1, average='binary', sample_weight=None)

0.734375

## Testing 10 extra contexts (Initial vectors + Tf idf + Random Forest)

In [504]:
vectorizer = TfidfVectorizer() # Tf idf

test_data = load_files('test_contexts.txt') # includes 10 extra contexts
test_data1 = [item[1] for item in test_data]

vectorized = vectorizer.fit_transform(test_data1)
print(vectorized.shape[0])

v = vectorized.toarray()

v1 = v[:1400]      # for training
v2 = v[1400:1410]  # 10 test sentences

clf = RandomForestClassifier(n_estimators=250)
y_labels = [item[0] for item in test_data[:1400]]

y_labels = [item[0] for item in test_data[:1400]]
X_train, X_test, y_train, y_test = train_test_split(v1, y_labels, test_size=0.1, random_state=42)

clf.fit(X_train, y_train)


1410


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=250, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [505]:
items = clf.predict(X_test)
right = 0
for i in range(len(items)):
    if items[i] == y_test[i]:
        right += 1
        
print ('Total: %d\nRight: %d\nScore: %.3f' % (len(items), right, float(right) / len(items)))

Total: 140
Right: 124
Score: 0.886


In [506]:
f1_score(y_test, items, labels=None, pos_label=1, average='binary', sample_weight=None)

0.90361445783132521

In [507]:
test_items = clf.predict(v2)
for i in test_items:
    print(i)

1
0
1
1
1
0
0
0
0
1


* Первые 5 примеров -- 1 (castle), последние 5 -- 0 (lock)
* <b>Результат: 8/10</b> (ошибочная классификация примеров 2 и 10):
* 2 -- 'ночью замок был освещен и был открыт для посетителей и ночных экскурсий'
* 10 -- 'ребенок пытался вставить карандаш в скважину замка и громко орал'